Para Maior contexto entre os modelos mais atuais também ajustei um Modelo Bert pré-treinado para comparação.  
No entanto modelos como Bert precisam de mais dados, Então precisei utilizar de versões mais completas dos   
 para tirar um Sample maior.  

DataSet já separado em train_test no kaggle:
https://www.kaggle.com/datasets/kritanjalijain/amazon-reviews

Toda as Versões dos Datasets disponibilizados também pela hugginface:
https://amazon-reviews-2023.github.io


Com isso saimos de 7000 samples para 72000, apenas 2% da base completa de 2013  com 34M de Reviews.  
 Com isso o modelo convergiu bem mais rápido e resultou
 claramente em um modelo substancialmente melhor:

| Model                | Accuracy |
|----------------------|----------|
| SVC                  | 0.8443   |
| Logistic Regression  | 0.8477   |
| Naive Bayes          | 0.8573   |
| Random Forest        | 0.8587   |
| LSTM                 | 0.8593   |
| BERT                 | 0.9348   |


In [1]:
# verifying version of transformers library

import numpy as np
import pandas as pd

import transformers
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, AdamW, Adagrad
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import joblib

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

from transformers import BertTokenizer, TFBertForSequenceClassification

In [10]:
import dask.dataframe as dd
# Load the data
df_test = dd.read_parquet('/content/test_sample.parquet')
df_train = dd.read_parquet('/content/train_sample.parquet')


In [3]:
df_test.head()

,rating,review_title,review_text
177219,2,LIFE SAVER for 1st Time Mother,I started reading this book when my son was 10...
231897,1,too few techicues for modeling.,"I liked the book overally, but it doesn't incl..."
87724,1,Item only worked for a short time.,I was glad to see that I could replace my old ...
344467,2,Wonderful for ALL girls!,I gave this to my niece for her 6th birthday. ...
323799,1,Lacking Depth,I was sadly disappinted when I read this book....


In [4]:
df_train.head()

,rating,review_title,review_text
673638,1,Way overpriced,OOPs. Don't make same mistake. I thought this ...
2419036,2,First half of book is stellar,I knew almost nothing about the growth of food...
497814,1,Boring,I was thrilled to pick this book up at a book ...
3594766,1,not the requested size,Apparently this is a good quality product. Any...
188400,2,"Lucky for me, these are still unopened",I keep two in each vehicle. Next time I purcha...


In [11]:
df_test['review_text'] = df_test['review_text'].astype('str')
df_test['rating'] = df_test['rating'].astype('int')
df_train['review_text'] = df_train['review_text'].astype('str')
df_train['rating'] = df_train['rating'].astype('int')


df_train = df_train.compute()
df_test = df_test.compute()


X_train = df_train['review_text'].values
y_train = df_train['rating'].values
X_test = df_test['review_text'].values
y_test = df_test['rating'].values

In [12]:
# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to tokenize texts
def tokenize_texts(texts, tokenizer, max_length=128):
    return tokenizer(
        texts.tolist(),
        max_length=max_length,
        padding=True,
        truncation=True,
        return_tensors='tf'
    )

# Tokenize the texts
tokenized_train_texts = tokenize_texts(X_train, tokenizer)
tokenized_test_texts = tokenize_texts(X_test, tokenizer)

# Encode target labels to one-hot
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
y_train_one_hot = tf.keras.utils.to_categorical(y_train_encoded, num_classes=2)  # Convert to one-hot encoding
y_test_one_hot = tf.keras.utils.to_categorical(y_test_encoded, num_classes=2)    # Convert to one-hot encoding

# Convert tokenized texts to NumPy arrays
X_train_input_ids = tokenized_train_texts['input_ids'].numpy()
X_train_attention_masks = tokenized_train_texts['attention_mask'].numpy()
X_test_input_ids = tokenized_test_texts['input_ids'].numpy()
X_test_attention_masks = tokenized_test_texts['attention_mask'].numpy()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 7.3 MB/s eta 0:00:00


In [14]:
# Function to train the BERT model
def train_bert(X_train, y_train, X_test, y_test, attention_masks_train, attention_masks_test):

    # Create the BERT model
    model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    model.compile(optimizer = 'SGD', loss=loss, metrics=['accuracy'])

    # Train the model
    model.fit(
        {'input_ids': X_train, 'attention_mask': attention_masks_train},
        y_train,
        validation_data=({'input_ids': X_test, 'attention_mask': attention_masks_test}, y_test),
        epochs=5,
        batch_size=16,
    )

    # Evaluate the model
    y_pred = np.argmax(model.predict({'input_ids': X_test, 'attention_mask': attention_masks_test}).logits, axis=1)
    y_test_labels = np.argmax(y_test, axis=1)
    classification_report_str = classification_report(y_test_labels, y_pred, target_names=label_encoder.classes_, zero_division=0)
    accuracy = accuracy_score(y_test_labels, y_pred)

    print("Classification Report:\n", classification_report_str)
    print("Accuracy:", accuracy)

    model.save_pretrained('big_bert_model')
    tokenizer.save_pretrained('big_bert_model')

    return classification_report_str, accuracy

In [ ]:
# Train the BERT model
report_bert, accuracy_bert = train_bert(X_train_input_ids, y_train_one_hot, X_test_input_ids, y_test_one_hot, X_train_attention_masks, X_test_attention_masks)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
4500/4500 [==============================] - 1943s 417ms/step - loss: 0.2339 - accuracy: 0.9052 - val_loss: 0.1793 - val_accuracy: 0.9279
Epoch 2/5
4500/4500 [==============================] - 1872s 416ms/step - loss: 0.1609 - accuracy: 0.9391 - val_loss: 0.1821 - val_accuracy: 0.9301
Epoch 3/5
4500/4500 [==============================] - 1885s 419ms/step - loss: 0.1240 - accuracy: 0.9543 - val_loss: 0.1804 - val_accuracy: 0.9319
Epoch 4/5
4500/4500 [==============================] - 1883s 418ms/step - loss: 0.0970 - accuracy: 0.9654 - val_loss: 0.1983 - val_accuracy: 0.9275
Epoch 5/5
4500/4500 [==============================] - 1881s 418ms/step - loss: 0.0764 - accuracy: 0.9726 - val_loss: 0.2088 - val_accuracy: 0.9348
